In [3]:
#loading file
%run dataset_loading.ipynb

In [76]:
#libraries
from scipy.signal import butter, filtfilt
import scipy.stats as ss

In [21]:
data = load_subject_data(17)['data'][1]

#### EEG rhythms:
$\alpha = 8-13 \text{Hz}, \ \ \beta = 13-30 \text{Hz},  \ \ \delta = 0.5-4 \text{Hz},  \ \ \theta = 4-8 \text{Hz},  \ \ \gamma = 30-60 \text{Hz}$ 

In [22]:
def bandpass_filter(signal, low_cutoff, high_cutoff, sample_rate, order=5):
    nyquist_freq = 0.5 * sample_rate
    low = low_cutoff / nyquist_freq
    high = high_cutoff / nyquist_freq
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

In [42]:
fs = 500
N = len(data) 
time = np.arange(N) / fs

In [45]:
alpha_rhythm = bandpass_filter(data, 8, 13, fs)
beta_rhythm = bandpass_filter(data, 13, 30, fs)
delta_rhythm = bandpass_filter(data, 0.5, 4, fs)
theta_rhythm = bandpass_filter(data, 4, 8, fs)
gamma_rhythm = bandpass_filter(data, 30, 60, fs)

In [47]:
plt.plot(time, data, label='Original')
plt.plot(time, gamma_rhythm, label='Filtered')
plt.legend()
plt.show()

In [63]:
def plot_spect(data, N, fs):
    freq_data = 2 * (np.fft.fft(data) / N)[:N//2+1]
    freqs = np.fft.fftfreq(N, 1/fs)[:N//2+1]
    plt.plot(freqs, np.abs(freq_data))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.title('One-Sided Spectrum')
    plt.xlim([0, 45])
    plt.show()

In [66]:
plot_spect(data, N, fs)
plot_spect(alpha_rhythm, N, fs)

#### Features

##### Mean

In [69]:
mean = np.mean(alpha_rhythm[0:10*fs])
mean

-0.00480592262461551

##### Variance

In [70]:
var = np.var(alpha_rhythm[0:10*fs])
var

16.21468091788232

##### STD

In [71]:
std = np.std(alpha_rhythm[0:10*fs])
std

4.026745698188839

##### Skewness

In [77]:
skew = ss.skew(alpha_rhythm[0:10*fs])
skew

0.003130516598097089

##### Kurtosis

In [79]:
kurt = ss.kurtosis(alpha_rhythm[0:10*fs])
kurt

0.9388739960816448